In [1]:
import pandas
import numpy
from matplotlib import pyplot as plt
import seaborn
%matplotlib notebook
import datetime
import general
import discretionary_aua
import vantage_aua
import data_accessing
import combined
import revenue
import costs
import consolidated
import discf
from ipywidgets import widgets
from IPython.display import display
import stats
pandas.set_option('display.float_format', '{:20,.4f}'.format)   #suppressing scientific notation
idx = pandas.IndexSlice

In [2]:
print('Pulling data...')
fund_dic = data_accessing.read_data(data_accessing.fund_data_name,data_accessing.fund_data_types)
nnb_dic = data_accessing.read_data(data_accessing.nnb_data_name,data_accessing.nnb_data_sheet)
discretionary_aua.append_share_class_units(fund_dic) # appending share class units
index_hl_dic = data_accessing.read_data(data_accessing.index_hl_data_name, data_accessing.index_hl_data_sheet)
data_dic = {**fund_dic, **index_hl_dic, **nnb_dic}

input_dic = data_accessing.read_data(data_accessing.assumptions_name, data_accessing.assumptions_sheet)

print('Data pulled successfully.')

Pulling data...
Data pulled successfully.


In [3]:
assumption_button = widgets.Button(description='Update assumptions',layout=widgets.Layout(width='20%',height='40px'))
assumption_button.style.button_color = 'lightgreen'
def update_assumption(b):
    global input_dic
    input_dic = data_accessing.read_data(data_accessing.assumptions_name, data_accessing.assumptions_sheet)
    print ('Assumptions updated successfully...')

assumption_button.on_click(update_assumption)
display(assumption_button)

In [4]:
revenue_types = ['Semi-annual revenue','Annual revenue - FY','Annual revenue - CY','Consolidated semi-annual revenue','Consolidated annual revenue - FY','Consolidated annual revenue - CY']
box_layout = widgets.Layout(display='flex', flex_flow='column', align_items='stretch',border='solid',width='30%')
items = {}
for w in revenue_types:
    items[w] = widgets.Button(description=w,layout=widgets.Layout(width='100%', height='40px'))
    items[w].style.button_color = 'lightgreen'
left_box = widgets.VBox([list(items.values())[0],list(items.values())[1],list(items.values())[2]],layout=box_layout)
right_box = widgets.VBox([list(items.values())[3],list(items.values())[4],list(items.values())[5]],layout=box_layout)
items['Semi-annual revenue'].on_click(lambda x: display(revenue.semi_revenue(data_dic,input_dic)))
items['Annual revenue - FY'].on_click(lambda x: display(revenue.annual_revenue(data_dic,input_dic)))
items['Annual revenue - CY'].on_click(lambda x: display(revenue.annual_revenue(data_dic,input_dic,cal_year=True)))
items['Consolidated semi-annual revenue'].on_click(lambda x: display(consolidated.revenue_analysis(data_dic,input_dic)))
items['Consolidated annual revenue - FY'].on_click(lambda x: display(consolidated.annual_revenue_analysis(data_dic,input_dic)))
items['Consolidated annual revenue - CY'].on_click(lambda x: display(consolidated.annual_revenue_analysis(data_dic,input_dic,cal_year=True)))
widgets.HBox([left_box,right_box])



In [5]:
costs_types = ['Semi-annual costs','Annual costs - FY','Annual costs - CY','Consolidated semi-annual costs','Consolidated annual costs - FY','Consolidated annual costs - CY']
items2 = {}
for w2 in costs_types:
    items2[w2] = widgets.Button(description=w2,layout=widgets.Layout(width='100%', height='40px'))
    items2[w2].style.button_color = 'lightgreen'
left_box2 = widgets.VBox([list(items2.values())[0],list(items2.values())[1],list(items2.values())[2]],layout=box_layout)
right_box2 = widgets.VBox([list(items2.values())[3],list(items2.values())[4],list(items2.values())[5]],layout=box_layout)
items2['Semi-annual costs'].on_click(lambda x: display(costs.semi_costs(input_dic)))
items2['Annual costs - FY'].on_click(lambda x: display(costs.annual_costs(input_dic)))
items2['Annual costs - CY'].on_click(lambda x: display(costs.annual_costs(input_dic,cal_year=True)))
items2['Consolidated semi-annual costs'].on_click(lambda x: display(consolidated.costs_analysis(input_dic)))
items2['Consolidated annual costs - FY'].on_click(lambda x: display(consolidated.costs_analysis(input_dic)))
items2['Consolidated annual costs - CY'].on_click(lambda x: display(consolidated.annual_costs_analysis(input_dic,cal_year=True)))
widgets.HBox([left_box2,right_box2])

,,,,,staff_costs,marketing_distribution,depre_amort_financial,office_running,FSCS_levy,others,capital_expenditure
month_end,financial_year,quarter_no,half_no,calendar_year,,,,,,,
2017-06-30,2017,4,2,2017,"-36,700,000.0000","-8,500,000.0000","-5,700,000.0000","-2,400,000.0000","-4,500,000.0000","-14,500,000.0000","-8,100,000.0000"
2017-12-29,2018,2,1,2017,"-40,993,900.0000","-8,755,000.0000","-5,871,000.0000","-2,832,000.0000","-2,250,000.0000","-14,858,150.0000","-8,300,070.0000"
2018-06-29,2018,4,2,2018,"-41,994,151.1600","-8,971,248.5000","-6,016,013.7000","-2,901,950.4000","-2,250,000.0000","-15,225,146.3050","-8,505,081.7290"
2018-12-31,2019,2,1,2018,"-43,031,406.6937","-9,192,838.3380","-6,164,609.2384","-2,973,628.5749","-2,250,000.0000","-15,601,207.4187","-8,715,157.2477"
2019-06-28,2019,4,2,2019,"-44,094,282.4390","-9,419,901.4449","-6,316,875.0866","-3,047,077.2007","-2,250,000.0000","-15,986,557.2420","-8,930,421.6317"
2019-12-31,2020,2,1,2019,"-45,183,411.2152","-9,652,573.0106","-6,472,901.9012","-3,122,340.0075","-2,250,000.0000","-16,381,425.2059","-9,151,003.0460"
2020-06-30,2020,4,2,2020,"-46,299,441.4722","-9,890,991.5639","-6,632,782.5782","-3,199,461.8057","-2,250,000.0000","-16,786,046.4084","-9,377,032.8213"
2020-12-31,2021,2,1,2020,"-47,443,037.6766","-10,135,299.0556","-6,796,612.3079","-3,278,488.5123","-2,250,000.0000","-17,200,661.7547","-9,608,645.5320"
2021-06-30,2021,4,2,2021,"-48,614,880.7072","-10,385,640.9423","-6,964,488.6319","-3,359,467.1786","-2,250,000.0000","-17,625,518.1001","-9,845,979.0766"


In [6]:
ops = [i for i in list(range(general.recent_end_year, general.recent_end_year+10))]
a_slider = widgets.SelectMultiple(options=ops, description='Years for comparison')
display(a_slider)

In [7]:
compare1 = ['NNB comparison','AUA comparison']
box_layout2 = widgets.Layout(display='flex', flex_flow='row', align_items='stretch',border='solid',width='30%')
items3 = {}
for w3 in compare1:
    items3[w3] = widgets.Button(description=w3,layout=widgets.Layout(width='100%', height='40px'))
    items3[w3].style.button_color = 'lightgreen'
box3 = widgets.Box([list(items3.values())[0],list(items3.values())[1]],layout=box_layout2)
items3['NNB comparison'].on_click(lambda x: display(consolidated.get_nnb_compare(data_dic, input_dic,list(a_slider.value))))
items3['AUA comparison'].on_click(lambda x: display(consolidated.get_aua_compare(data_dic, input_dic,list(a_slider.value))))


display(box3)

Vantage nnb              PMS nnb  \
financial_year quarter_no half_no                                             
2017           1          1         1,098,900,000.0000      11,100,000.0000   
               2          1         1,217,700,000.0000      12,300,000.0000   
               3          2         3,267,000,000.0000      33,000,000.0000   
               4          2         1,249,626,000.0000      10,395,000.0000   
2018           1          1         1,534,340,500.0000       5,678,750.0000   
               2          1         1,261,065,757.8462       9,369,223.7587   
               3          2         3,527,354,900.2083      26,206,862.8308   
               4          2         2,359,320,512.5482      17,528,825.6486   
2019           1          1         1,441,553,661.2194      10,710,178.0602   
               2          1         1,455,757,875.5056      10,815,709.7989   
               3          2         3,144,384,939.6143      23,361,546.2949   
               4          2         2,103,165,714.0430      15,625,696.0067   

                                        HLMM Funds nnb     Cash Service nnb  \
financial_year quarter_no half_no                                             
2017           1          1           177,600,000.0000               0.0000   
               2          1           196,800,000.0000               0.0000   
               3          2           528,000,000.0000               0.0000   
               4          2           176,421,000.0000               0.0000   
2018           1          1           100,119,250.0000               0.0000   
               2          1           139,776,114.4482               0.0000   
               3          2           390,970,858.7069               0.0000   
               4          2           261,506,310.7773               0.0000   
2019           1          1           159,781,334.3834               0.0000   
               2          1           161,355,724.8300               0.0000   
               3          2           348,522,594.0473               0.0000   
               4          2           233,114,197.0358               0.0000   

                                             Total nnb  
financial_year quarter_no half_no                       
2017           1          1         1,110,000,000.0000  
               2          1         1,230,000,000.0000  
               3          2         3,300,000,000.0000  
               4          2         1,260,021,000.0000  
2018           1          1         1,540,019,250.0000  
               2          1         1,270,434,981.6049  
               3          2         3,553,561,763.0392  
               4          2         2,376,849,338.1968  
2019           1          1         1,452,263,839.2797  
               2          1         1,466,573,585.3045  
               3          2         3,167,746,485.9092  
               4          2         2,118,791,410.0497

Actual                       \
                                Vantage AUA              PMS AUA   
financial_year half_no                                             
2017           1        66,792,000,000.0000   3,209,000,000.0000   
               2        75,800,000,000.0000   3,400,000,000.0000   
2018           1                        nan                  nan   
               2                        nan                  nan   
2019           1                        nan                  nan   
               2                        nan                  nan   

                                                                  \
                             HLMM Funds AUA     Cash Service AUA   
financial_year half_no                                             
2017           1         7,581,000,000.0000               0.0000   
               2         8,800,000,000.0000               0.0000   
2018           1                        nan                  nan   
               2                        nan                  nan   
2019           1                        nan                  nan   
               2                        nan                  nan   

                                                     WP forecast  \
                                  Total AUA          Vantage AUA   
financial_year half_no                                             
2017           1        70,001,000,000.0000                  nan   
               2        79,200,000,000.0000  75,800,000,000.0000   
2018           1                        nan  83,847,131,627.5090   
               2                        nan  91,394,307,478.2468   
2019           1                        nan  95,985,003,842.0910   
               2                        nan 102,959,474,951.2488   

                                                                  \
                                    PMS AUA       HLMM Funds AUA   
financial_year half_no                                             
2017           1                        nan                  nan   
               2         3,400,000,000.0000   8,800,000,000.0000   
2018           1         3,482,684,553.7964   9,215,786,607.4384   
               2         3,595,390,988.3786  10,050,772,318.3713   
2019           1         3,687,253,512.2868  10,558,032,300.4335   
               2         3,797,970,330.5353  11,329,477,974.6233   

                                                                  
                           Cash Service AUA            Total AUA  
financial_year half_no                                            
2017           1                        nan                  nan  
               2                     0.0000  79,200,000,000.0000  
2018           1                     0.0000  87,329,816,181.3054  
               2                     0.0000  94,989,698,466.6254  
2019           1           300,000,000.0000  99,972,257,354.3777  
               2           750,000,000.0000 107,507,445,281.7841

In [8]:
compare2 = ['Interim revenue comparison','Annual revenue comparison', 'Interim costs comparison', 'Annual costs comparison']
box_layout3 = widgets.Layout(display='flex', flex_flow='row', align_items='stretch',border='solid',width='50%')
items4 = {}
for w4 in compare2:
    items4[w4] = widgets.Button(description=w4,layout=widgets.Layout(width='100%', height='40px'))
    items4[w4].style.button_color = 'lightgreen'
top_box4 = widgets.HBox([list(items4.values())[0],list(items4.values())[1]],layout=box_layout3)
bottom_box4 = widgets.HBox([list(items4.values())[2],list(items4.values())[3]],layout=box_layout3)
items4['Interim revenue comparison'].on_click(lambda x: display(consolidated.get_revenue_compare(data_dic, input_dic,True,list(a_slider.value))))
items4['Annual revenue comparison'].on_click(lambda x: display(consolidated.get_revenue_compare(data_dic, input_dic,False,list(a_slider.value))))
items4['Interim costs comparison'].on_click(lambda x: display(consolidated.get_costs_compare(input_dic,True,list(a_slider.value))))
items4['Annual costs comparison'].on_click(lambda x: display(consolidated.get_costs_compare(input_dic,False,list(a_slider.value))))

widgets.VBox([top_box4, bottom_box4])

In [9]:
'''=====current values=======
discounted cash flow period: 9 years
perpetuity growth rate: 0.035
cash flow discounted rate: 0.085
'''
dcf_period = widgets.IntText(description='DCF period - No. of years:')
display(dcf_period)
perp_growth = widgets.FloatText(description='Perpetuity growth rate:')
display(perp_growth)
disct_rate = widgets.FloatText(description='Cash flow discount rate:')
display(disct_rate)
current_value = widgets.Checkbox(value=False, description='Fair value now')
display(current_value)

In [10]:
dis_cash = ['Discounted cash flow','Fair value']
items5 = {}
for w5 in dis_cash:
    items5[w5] = widgets.Button(description=w5, layout=widgets.Layout(width='100%', height='40px'))
    items5[w5].style.button_color = 'lightgreen'
fair = widgets.HBox([list(items5.values())[0],list(items5.values())[1]],layout=box_layout3)
items5['Discounted cash flow'].on_click(lambda x: display(discf.disc_cash_flow(data_dic, input_dic,now=current_value.value,dcf_p=dcf_period.value,disc_rate=disct_rate.value)))
items5['Fair value'].on_click(lambda x: display(discf.fair_value(data_dic, input_dic,now=current_value.value,dcf_p=dcf_period.value,disc_rate=disct_rate.value,pep_rate=perp_growth.value)))    
display(fair)

,HL
Terminal value,"11,734,244,496.7415"
Enterprise value,"7,802,696,157.9444"
Net debt&cash,114157118
Fair value,"7,916,853,275.9444"
No. of shares,474402854
Fair value per share,16.6880


,HL
Terminal value,"12,913,390,311.2754"
Enterprise value,"8,644,139,151.6389"
Net debt&cash,114157118
Fair value,"8,758,296,269.6389"
No. of shares,474402854
Fair value per share,18.4617


,HL
Terminal value,"14,417,526,488.1658"
Enterprise value,"9,365,943,531.8346"
Net debt&cash,114157118
Fair value,"9,480,100,649.8346"
No. of shares,474402854
Fair value per share,19.9832


In [12]:
others = ['HLF composite monthly return','Total NNB','NNB distribution','Total AUA','% Revenue distribution','Avg AUA distribution']
items6 = {}
for w6 in others:
    items6[w6] = widgets.Button(description=w6, layout=widgets.Layout(width='100%', height='40px'))
    items6[w6].style.button_color = 'lightgreen'
other_box1 = widgets.VBox([list(items6.values())[0],list(items6.values())[1],list(items6.values())[2]],layout=box_layout)
other_box2 = widgets.VBox([list(items6.values())[3],list(items6.values())[4],list(items6.values())[5]],layout=box_layout)
items6['HLF composite monthly return'].on_click(lambda x: display(discretionary_aua.get_composite_return(data_dic)))
items6['Total NNB'].on_click(lambda x: display(combined.total_nnb(data_dic, input_dic)))
items6['NNB distribution'].on_click(lambda x: display(combined.nnb_distribution(data_dic, input_dic)))
items6['Total AUA'].on_click(lambda x: display(combined.total_aua(data_dic, input_dic)))
items6['% Revenue distribution'].on_click(lambda x: display(stats.summary_revenue_dist_percent(data_dic, input_dic)))
items6['Avg AUA distribution'].on_click(lambda x: display(stats.summary_avg_aua_dist(data_dic, input_dic)))


widgets.HBox([other_box1, other_box2])

#Financial year based

In [13]:
nnb_radio = widgets.RadioButtons(options=['day','month','quarter','annual'],description='Period',disabled=False)
nnb_dropdown = widgets.Dropdown(options=['no_select',None],description='Fund option',disabled=False)
display(nnb_radio)
display(nnb_dropdown)

nnb_b = widgets.Button(description='Implied HLF nnb period-to-date', layout=widgets.Layout(width='50%', height='40px'))
nnb_b.style.button_color = 'lightgreen'
nnb_b.on_click(lambda x: display(stats.hlf_to_date_implied_nnb(data_dic,typ=nnb_radio.value,fund_opt=nnb_dropdown.value)))
display(nnb_b)

HLF nnb
financial_year quarter_no                     
2015           2                        0.0000
               3                        0.0000
               4                        0.0000
2016           1                        0.0000
               2             -278,532,004.8059
               3              458,288,241.8552
               4               55,170,068.7077
2017           1               64,377,424.1206
               2             -109,158,613.4505
               3              501,770,003.2071
               4              222,940,072.3881
2018           1              123,741,695.3698
               2              159,583,882.7467
               3               26,508,908.9628
               4                        0.0000
2019           1                        0.0000

In [14]:
cash_radio = widgets.RadioButtons(options=['monthly','quarter_no','half_no','financial_year','calendar_year'],description='Period',disabled=False)
display(cash_radio)


cash_b = widgets.Button(description='Cash margin', layout=widgets.Layout(width='50%', height='40px'))
cash_b.style.button_color = 'lightgreen'
cash_b.on_click(lambda x: display(stats.cash_margin(data_dic,cash_radio.value)))
display(cash_b)

In [15]:
eps_radio = widgets.RadioButtons(options=[True, False],description='Calendar year?',disabled=False)
display(eps_radio)

eps_summary = widgets.Button(description='Net earnings and EPS', layout=widgets.Layout(width='50%', height='40px'))
eps_summary.style.button_color = 'lightgreen'
eps_summary.on_click(lambda x: display(stats.summary_total(data_dic, input_dic, cal=eps_radio.value)))
display(eps_summary)


In [16]:
# %load_ext autoreload
# %autoreload 2